In [13]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import statsmodels
import matplotlib.pyplot as plt
import seaborn
import dateutil
from datetime import datetime
import math
import plotly.graph_objects as go
import plotly.express as px
import math
import functions.functions as base_fx
import functions.portfolio_functions as portfolio_fx
import functions.algorithms as algos
from statsmodels.tsa.stattools import coint

import importlib


In [14]:
importlib.reload(base_fx)
importlib.reload(portfolio_fx)
importlib.reload(algos)


<module 'functions.algorithms' from '../functions/algorithms.py'>

# Get Data

In [19]:
quotes = pd.DataFrame()
#start = '2018-01-1'
start = '2016-01-1'
end = '2021-08-25'
symbols = ['LUV.US','GD.US']

In [20]:
for s in symbols:
    print(s)
    endpoint = f'https://eodhistoricaldata.com/api/eod/{s}?from={start}&to={end}&api_token=5d5a6c41cd6592.15989098&fmt=csv'
    df = pd.read_csv(endpoint)
    #print(df.head(1))
    df = df[df['Adjusted_close'] > 0]
    #print(df.head(1))
    if len(quotes) == 0:
        quotes['Date'] = df['Date']
        quotes[s] = df['Adjusted_close']
    else:
        new_df = pd.DataFrame()
        new_df['Date'] = df['Date']
        new_df[s] = df['Adjusted_close']
        quotes = quotes.merge(new_df, how="inner", on="Date")
   

LUV.US
GD.US


In [21]:
# backup quotes
old_quotes = quotes

In [22]:
quotes = quotes.dropna()
quotes = quotes.reset_index()
quotes['Date'] = pd.to_datetime(quotes['Date'])
quotes = quotes.set_index('Date')


# Params

In [23]:
symbol1 = 'LUV.US'
symbol2 = 'GD.US'

In [24]:
# Indicators
macd_fast_points = 1
macd_slow_points = 2
max_leverage = 2

# Could be changed
mavg_1 = 5 # default 5
mavg_2 = 60 #default 60

## Optimize trading limits looking at zscore
limit = 0.75 #default 1
exit_limit = 0.5 #default -.75
leverage_limit = 1
zscore_limit = 1.5

## Optimize these with bruteforce
vol_interval = 30
vol_peak_period = 365 # 1 year
vol_mean_peak_period = 60 # 2 months
coint_limit =  1
coint_period = 365


In [25]:
c = coint(quotes[symbol1], quotes[symbol2])
print(c[1])

0.05370517774588988


# Trade

In [26]:
settings = {
    'symbol1': symbol1,
    'symbol2': symbol2,
    'mavg_1' : mavg_1,
    'mavg_2': mavg_2,
    'macd_fast_points': macd_fast_points,
    'macd_slow_points': macd_slow_points,
    'vol_interval': vol_interval,
    'vol_peak_period': vol_peak_period,
    'vol_mean_peak_period': vol_mean_peak_period,
    'limit': limit,
    'leverage_limit': leverage_limit,
    'exit_limit': exit_limit,
    'max_leverage': max_leverage,
    'coint_limit': coint_limit,
    'coint_period': coint_period,
    'sl_limit': -0.05
}

In [29]:
importlib.reload(base_fx)
importlib.reload(portfolio_fx)
importlib.reload(algos)
portfolio, mavg_zscore = algos.pair_trade_with_zscore_limit(settings, quotes)

TypeError: loop of ufunc does not support argument 0 of type float which has no callable log method

In [30]:
portfolio_fx.plot_trades_on_zscore_for_symbol(mavg_zscore, portfolio, symbol1)
portfolio_fx.plot_trades_on_zscore_for_symbol(mavg_zscore, portfolio, symbol2)


NameError: name 'mavg_zscore' is not defined

In [31]:
importlib.reload(base_fx)
importlib.reload(portfolio_fx)
importlib.reload(algos)
#print(quotes)
portfolio_fx.plot_trades_on_series(symbol1, symbol2, quotes[symbol1], quotes[symbol2], portfolio)


NameError: name 'portfolio' is not defined